In [1]:
import pandas as pd
from datetime import date
from datetime import timedelta
from datetime import *
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

1- Reading the data

In [2]:
df = pd.read_csv("energy_Data.csv")

In [3]:
df

,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST,NOTES
0,Electric usage,10/22/2016,0:00,0:14,0.01,kWh,$0.00,NaN
1,Electric usage,10/22/2018,0:15,0:29,0.01,kWh,$0.00,NaN
2,Electric usage,10/22/2016,0:30,0:44,NaN,kWh,$0.00,NaN
3,Electric usage,10/22/2016,0:45,0:59,NaN,kWh,$0.00,NaN
4,Electric usage,10/22/2016,1:00,1:14,0.03,kWh,$0.00,NaN
...,...,...,...,...,...,...,...,...
70364,Electric usage,10/24/2018,23:00,23:14,0.03,kWh,$0.01,NaN
70365,Electric usage,10/24/2018,23:15,23:29,0.03,kWh,$0.01,NaN
70366,Electric usage,10/24/2018,23:30,23:44,0.03,kWh,$0.01,NaN
70367,Electric usage,10/24/2018,23:45,23:59,0.03,kWh,$0.01,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70369 entries, 0 to 70368
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TYPE        70369 non-null  object 
 1   DATE        70369 non-null  object 
 2   START TIME  70369 non-null  object 
 3   END TIME    70369 non-null  object 
 4   USAGE       70366 non-null  float64
 5   UNITS       70369 non-null  object 
 6   COST        70369 non-null  object 
 7   NOTES       0 non-null      float64
dtypes: float64(2), object(6)
memory usage: 4.3+ MB


Droping unneccessary columns.

In [5]:
#Since Notes column is empty we can drop it.

del df["NOTES"]
df

,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST
0,Electric usage,10/22/2016,0:00,0:14,0.01,kWh,$0.00
1,Electric usage,10/22/2018,0:15,0:29,0.01,kWh,$0.00
2,Electric usage,10/22/2016,0:30,0:44,NaN,kWh,$0.00
3,Electric usage,10/22/2016,0:45,0:59,NaN,kWh,$0.00
4,Electric usage,10/22/2016,1:00,1:14,0.03,kWh,$0.00
...,...,...,...,...,...,...,...
70364,Electric usage,10/24/2018,23:00,23:14,0.03,kWh,$0.01
70365,Electric usage,10/24/2018,23:15,23:29,0.03,kWh,$0.01
70366,Electric usage,10/24/2018,23:30,23:44,0.03,kWh,$0.01
70367,Electric usage,10/24/2018,23:45,23:59,0.03,kWh,$0.01


Converting time based columns to datetime object.

In [6]:
# We need to change the date columns from object to datetime.

df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y')
df['START TIME'] = pd.to_datetime(df['START TIME'], format='%H:%M')
df['END TIME'] = pd.to_datetime(df['END TIME'], format='%H:%M')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70369 entries, 0 to 70368
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   TYPE        70369 non-null  object        
 1   DATE        70369 non-null  datetime64[ns]
 2   START TIME  70369 non-null  datetime64[ns]
 3   END TIME    70369 non-null  datetime64[ns]
 4   USAGE       70366 non-null  float64       
 5   UNITS       70369 non-null  object        
 6   COST        70369 non-null  object        
dtypes: datetime64[ns](3), float64(1), object(3)
memory usage: 3.8+ MB


Modifying the time based columns.

In [8]:
df['START TIME'] = df['START TIME'].dt.strftime('%H:%M:%S')
df['END TIME'] = df['END TIME'].dt.strftime('%H:%M:%S')
df

,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST
0,Electric usage,2016-10-22,00:00:00,00:14:00,0.01,kWh,$0.00
1,Electric usage,2018-10-22,00:15:00,00:29:00,0.01,kWh,$0.00
2,Electric usage,2016-10-22,00:30:00,00:44:00,NaN,kWh,$0.00
3,Electric usage,2016-10-22,00:45:00,00:59:00,NaN,kWh,$0.00
4,Electric usage,2016-10-22,01:00:00,01:14:00,0.03,kWh,$0.00
...,...,...,...,...,...,...,...
70364,Electric usage,2018-10-24,23:00:00,23:14:00,0.03,kWh,$0.01
70365,Electric usage,2018-10-24,23:15:00,23:29:00,0.03,kWh,$0.01
70366,Electric usage,2018-10-24,23:30:00,23:44:00,0.03,kWh,$0.01
70367,Electric usage,2018-10-24,23:45:00,23:59:00,0.03,kWh,$0.01


Sorting time based columns.

In [9]:
# We need to sort df according to our datetime column. 
# It can be unstationary. Also we should sort accordingly 
# to start time. 

df.sort_values(by=['DATE',"START TIME"], inplace = True)

df

,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST
0,Electric usage,2016-10-22,00:00:00,00:14:00,0.01,kWh,$0.00
2,Electric usage,2016-10-22,00:30:00,00:44:00,NaN,kWh,$0.00
3,Electric usage,2016-10-22,00:45:00,00:59:00,NaN,kWh,$0.00
4,Electric usage,2016-10-22,01:00:00,01:14:00,0.03,kWh,$0.00
5,Electric usage,2016-10-22,01:15:00,01:29:00,NaN,kWh,$0.00
...,...,...,...,...,...,...,...
70364,Electric usage,2018-10-24,23:00:00,23:14:00,0.03,kWh,$0.01
70365,Electric usage,2018-10-24,23:15:00,23:29:00,0.03,kWh,$0.01
70366,Electric usage,2018-10-24,23:30:00,23:44:00,0.03,kWh,$0.01
70367,Electric usage,2018-10-24,23:45:00,23:59:00,0.03,kWh,$0.01


Handling the missing data.

In [10]:
# For handling missing values we can use ffill 
# and bfill functions. 

filled_column = pd.concat([df.ffill(), 
                df.bfill()]).groupby(level=0).mean()
df["USAGE"] = filled_column 
df

,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST
0,Electric usage,2016-10-22,00:00:00,00:14:00,0.01,kWh,$0.00
2,Electric usage,2016-10-22,00:30:00,00:44:00,0.02,kWh,$0.00
3,Electric usage,2016-10-22,00:45:00,00:59:00,0.02,kWh,$0.00
4,Electric usage,2016-10-22,01:00:00,01:14:00,0.03,kWh,$0.00
5,Electric usage,2016-10-22,01:15:00,01:29:00,0.02,kWh,$0.00
...,...,...,...,...,...,...,...
70364,Electric usage,2018-10-24,23:00:00,23:14:00,0.03,kWh,$0.01
70365,Electric usage,2018-10-24,23:15:00,23:29:00,0.03,kWh,$0.01
70366,Electric usage,2018-10-24,23:30:00,23:44:00,0.03,kWh,$0.01
70367,Electric usage,2018-10-24,23:45:00,23:59:00,0.03,kWh,$0.01


Droping the duplicatees.

In [11]:
# Check if there are duplicates in dataframe. Delete if it has.

if ((df.duplicated()).sum() > 0):
    print("There are:",(df.duplicated()).sum(), "duplicates.")
    df.drop_duplicates(inplace=True)
    

df

There are: 1 duplicates.


,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST
0,Electric usage,2016-10-22,00:00:00,00:14:00,0.01,kWh,$0.00
2,Electric usage,2016-10-22,00:30:00,00:44:00,0.02,kWh,$0.00
3,Electric usage,2016-10-22,00:45:00,00:59:00,0.02,kWh,$0.00
4,Electric usage,2016-10-22,01:00:00,01:14:00,0.03,kWh,$0.00
5,Electric usage,2016-10-22,01:15:00,01:29:00,0.02,kWh,$0.00
...,...,...,...,...,...,...,...
70363,Electric usage,2018-10-24,22:45:00,22:59:00,0.02,kWh,$0.00
70364,Electric usage,2018-10-24,23:00:00,23:14:00,0.03,kWh,$0.01
70365,Electric usage,2018-10-24,23:15:00,23:29:00,0.03,kWh,$0.01
70366,Electric usage,2018-10-24,23:30:00,23:44:00,0.03,kWh,$0.01


Outlier detections.

In [12]:
# Outlier Detection

# Z-Score method

z_scores = np.abs(stats.zscore(df['USAGE']))
outliers = (np.where(z_scores > 3))

print("There are:", len(outliers[0]), "outliers.")

There are: 1460 outliers.


In [13]:
# Iqr method

Q1 = np.percentile(df['USAGE'], 3,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(df['USAGE'], 97,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
 
# Upper bound
upper = np.where(df['USAGE'] >= (Q3+1.5*IQR))
# Lower bound
lower = np.where(df['USAGE'] <= (Q1-1.5*IQR))
 
print("There are:", len(upper[0]), "outliers in upper bound.")
print("There are:", len(lower[0]), "outliers in upper bound.")

There are: 556 outliers in upper bound.
There are: 0 outliers in upper bound.


Scaling the columns.

In [14]:
# Scaling the column 'USAGE'

print("Max value for Usage is:" ,df["USAGE"].max())
print("Min value for Usage is:" ,df["USAGE"].min())

scaler = MinMaxScaler()
df['Usage_scaled'] = scaler.fit_transform(df['USAGE'].values.reshape(-1,1))
df

Max value for Usage is: 2.36
Min value for Usage is: 0.0


,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST,Usage_scaled
0,Electric usage,2016-10-22,00:00:00,00:14:00,0.01,kWh,$0.00,0.004237
2,Electric usage,2016-10-22,00:30:00,00:44:00,0.02,kWh,$0.00,0.008475
3,Electric usage,2016-10-22,00:45:00,00:59:00,0.02,kWh,$0.00,0.008475
4,Electric usage,2016-10-22,01:00:00,01:14:00,0.03,kWh,$0.00,0.012712
5,Electric usage,2016-10-22,01:15:00,01:29:00,0.02,kWh,$0.00,0.008475
...,...,...,...,...,...,...,...,...
70363,Electric usage,2018-10-24,22:45:00,22:59:00,0.02,kWh,$0.00,0.008475
70364,Electric usage,2018-10-24,23:00:00,23:14:00,0.03,kWh,$0.01,0.012712
70365,Electric usage,2018-10-24,23:15:00,23:29:00,0.03,kWh,$0.01,0.012712
70366,Electric usage,2018-10-24,23:30:00,23:44:00,0.03,kWh,$0.01,0.012712
